In [1]:
from hyrax import Hyrax

h = Hyrax()

[2025-04-21 21:05:18,058 hyrax:INFO] Runtime Config read from: /home/drew/code/hyrax/src/hyrax/hyrax_default_config.toml


Quickly train a model using the default CiFAR dataset and then use the model to run inference.

In [2]:
h.config["train"]["epochs"] = 1
h.train()
h.infer()

[2025-04-21 21:05:25,772 hyrax.models.model_registry:INFO] Using criterion: torch.nn.CrossEntropyLoss with default arguments.
2025-04-21 21:05:26,049 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset HyraxCifarDa': 
	{'sampler': <hyrax.pytorch_ignite.SubsetSequentialSampler object at 0x7fef746e7d10>, 'batch_size': 512, 'shuffle': False, 'pin_memory': True}
2025-04-21 21:05:26,051 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset HyraxCifarDa': 
	{'sampler': <hyrax.pytorch_ignite.SubsetSequentialSampler object at 0x7fef746e7d70>, 'batch_size': 512, 'shuffle': False, 'pin_memory': True}
/home/drew/miniconda3/envs/hyrax/lib/python3.12/site-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2025/04/21 21:05:26 INFO mlflow.system_metrics.

  2%|1         | 1/59 [00:00<?, ?it/s]

State:
	iteration: 59
	epoch: 1
	epoch_length: 59
	max_epochs: 1
	output: <class 'dict'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>



[2025-04-21 21:05:35,201 hyrax.pytorch_ignite:INFO] Total training time: 8.73[s]
[2025-04-21 21:05:35,202 hyrax.pytorch_ignite:INFO] Latest checkpoint saved as: /home/drew/code/hyrax/docs/pre_executed/results/20250421-210518-train-Z8B5/checkpoint_epoch_1.pt
[2025-04-21 21:05:35,203 hyrax.pytorch_ignite:INFO] Best metric checkpoint saved as: /home/drew/code/hyrax/docs/pre_executed/results/20250421-210518-train-Z8B5/checkpoint_1_loss=-349.7285.pt
2025/04/21 21:05:35 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/04/21 21:05:35 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[2025-04-21 21:05:35,217 hyrax.train:INFO] Finished Training
[2025-04-21 21:05:35,569 hyrax.model_exporters:INFO] Exported model to ONNX format: /home/drew/code/hyrax/docs/pre_executed/results/20250421-210518-train-Z8B5/example_model_opset_20.onnx
[2025-04-21 21:05:43,388 hyrax.models.model_registry:INFO] Using criteri

  1%|1         | 1/98 [00:00<?, ?it/s]

[2025-04-21 21:05:53,944 hyrax.pytorch_ignite:INFO] Total evaluation time: 10.29[s]
[2025-04-21 21:05:54,012 hyrax.verbs.infer:INFO] Inference Complete.
[2025-04-21 21:05:54,296 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.
[2025-04-21 21:05:54,297 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.
[2025-04-21 21:05:57,277 hyrax.config_utils:WARNING] Cannot find default_config.toml for umap.


Use the output of inference to populate a ChromaDB vector database.

In [3]:
h.index()

[2025-04-21 21:06:05,033 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.
[2025-04-21 21:06:05,034 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.
[2025-04-21 21:06:05,034 hyrax.config_utils:WARNING] Cannot find default_config.toml for umap.
[2025-04-21 21:06:12,843 hyrax.verbs.vdb_index:INFO] Number of inference result batches to index: 98.
100%|██████████| 98/98 [00:09<00:00, 10.68it/s]


Some examples of simple database operations. Here we get the directory that contains the vector database that was just populated and create an instance of the ChromaDB object for querying.

In [ ]:
from hyrax.vector_dbs import ChromaDB
from hyrax.config_utils import find_most_recent_results_dir

context = {"results_dir": find_most_recent_results_dir(h.config, "index")}
cdb = ChromaDB({}, context)

Next we'll retrieve the vectors associated with two ids. The result is a dictionary where the keys are id strings and values are the numpy array vectors stored in the vector database.

In [21]:
results = cdb.get_by_id(ids=["97", "334"])

for k, v in results.items():
    print(f"Id: {k}, Vector: {v}")

Id: 334, Vector: [-0.97002649  1.38855457 -0.25365093  1.69827247  0.30844495 -0.12353173
 -2.88466191 -2.90000081 -4.31662893 -2.28196621  0.9391892  -2.26396465
  0.85602278  2.23173761 -0.24233057  0.02056995  1.27067137  0.02066563
  3.0384562   2.72503877 -2.39113975 -0.69743693  1.5756855   2.77459621
  0.77587169 -1.50103259 -0.59780228  1.90591443  2.80806208 -0.05698622
 -0.1125446   0.69265711 -0.23092759  2.14646101  0.07940467  2.89812326
  1.65633786  1.97285414  0.87678236 -2.04588151  0.3986209  -0.8836394
 -2.12579894  1.34612143 -0.48207313  0.4251518  -0.10547816 -3.16599846
 -2.07017779  2.21695733 -0.90912127 -1.21102476  0.57595026 -1.40068698
  3.16039634  1.31321025  2.21002913 -0.91437978 -1.51677895  0.22025201
  2.69880056 -1.87182248  4.19583607  0.13765451]
Id: 97, Vector: [ 0.08690037  5.78720379  1.33209431  3.88893032 -3.15039277 -1.7888478
 -2.6906569  -1.77430046 -3.68156838  1.25691974  1.95259762 -2.50791049
 -0.60998529  3.41917467 -2.52410293 -0.816

Here we'll search for the 5 nearest neighbors of the vector with id = '334'. Note that the closest of the 5 neighbors is the vector itself, since it's in the database.

In [ ]:
cdb.search_by_id("334", k=5)

{0: ['334', '3324', '21558', '48380', '35536']}

We can repeat the search, using the vector that we found when running `get_by_id`. The results from this search should match those of `search_by_id`.

In [15]:
cdb.search_by_vector([results["334"]], k=5)

{0: ['334', '3324', '21558', '48380', '35536']}